**PREAMBLE**

In [ ]:
import splashback.cluster
reload(splashback.cluster)
cluster_sample = splashback.cluster.cluster_sample

import numpy as np
import emcee
from getdist import plots, MCSamples
import corner

from matplotlib import pyplot as plt
from astropy import units as u
from splashback.profile import DK14, rho, rho_infall, rho_Ein, f_trans
%matplotlib inline
plt.rc('font', size=20)

**VARIABLES**

In [ ]:
n_disc = 100 # burn-in
chain_name = 'highm' 
r_limit = True
names = ['rho_s', 'r_s', 'logalpha', 'log r_t', 'logbeta', 'loggamma', 'rho_0', 's_e']
labels =  [r'$\rho_s$', 'r_s', r'$\log \alpha$', '$\log r_t$', r'$\log \beta$', r'$\log \gamma$', r"$\rho_0$", "$s_e$"]

n_samples = 1000# How many points to take for the posteriors in r_splash and co


filename = "output/chains/"+chain_name+'_'+str(r_limit)+'.h5'
if(chain_name=='simulation'):
    filename = "output/chains/simulation.h5"
backend = emcee.backends.HDFBackend(filename, read_only=True)

data = backend.get_chain()

**INSPECT CHAINS**

In [ ]:
print chain_name
data = backend.get_chain()
#print backend.get_autocorr_time(discard=n_disc)
for i in xrange(8):
    plt.figure()
    plt.gca().set_xlabel('chain step')
    plt.gca().set_ylabel(r"Position of "+names[i])
    plt.axvline(n_disc, c='r', ls='--')
    plt.plot(data[:, :, i], c='k', alpha=0.1)
    

lnprob = backend.get_log_prob(discard=n_disc)

plt.figure()
plt.gca().set_xlabel('chain step')
plt.gca().set_ylabel(r"max $\log \mathcal{L}$")
plt.plot(lnprob[:, :].max(axis=1), c='k', alpha=0.1)
plt.gca().set_ylim([-10, 0])
plt.show()

**TRIANGLE PLOT**

In [ ]:
print chain_name
samps = backend.get_chain(flat = True, discard=n_disc)
#points = MCSamples(samples=samps,names = names, labels = labels)
#g = plots.getSubplotPlotter()
#g.triangle_plot(points, filled=True)
corner.corner(samps, labels=labels, show_titles=True, plot_datapoints=False, plot_density=False, quantiles=(0.16, 0.84), levels=(1-np.exp(-0.5),1-np.exp(-2)), fill_contours=True)#, truths=params_best)
plt.show()


**SPLASHBACK POSITION**

In [ ]:
print chain_name

backend = emcee.backends.HDFBackend(filename, read_only=True)
data = backend.get_chain(flat = True, discard=n_disc)
params = data[np.random.choice(data.shape[0], n_samples), :]
r = np.linspace(.3, 70., 1000)
logr = np.log(r)


r_sp = np.zeros(n_samples)

der = np.zeros(n_samples)
der_c = np.zeros(n_samples)
fraction_at_0 = np.zeros(n_samples)
der_E = np.zeros(n_samples)
der_onesigma = np.zeros((n_samples, len(r)))
der_c_onesigma = np.zeros((n_samples, len(r)))
plt.figure(figsize=(10, 10))

for i in xrange(n_samples):
    rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params[i, :]
    #Total derivative 
    para_tot = [rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e]
    y_tot = rho(r, para_tot)
    logy_tot = np.log(y_tot)
    derivative = np.gradient(logy_tot, logr)
    plt.plot(r, derivative, c='k', alpha=0.01)
    der_onesigma[i] = derivative
    
    # Plot fraction of collapsed material 
    y_c = rho_Ein(r, [rho_s, r_s, logalpha])*f_trans(r, [r_t, logbeta, loggamma])
    logy_c = np.log(y_c)
    
    y_i = rho_infall(r, [rho_0, s_e])
    
    derivative = np.gradient(logy_tot, logr)
    derivative_collapsed = np.gradient(logy_c, logr)
    
    r_sp[i] = r[derivative.argmin()]
    der[i] = derivative.min()
    der_c[i] = derivative_collapsed[derivative.argmin()]
    
    fraction_at_0[i] = y_c[0]/y_tot[0]
    
y_plotn2 = -1.*(1.+3.*r/0.5)/(1.+r/0.5)
plt.plot(r, y_plotn2, '--b')
#plt.axhline(-2.5, c='r', alpha=0.5)
plt.plot(r, np.percentile(der_onesigma, 15.86, axis=0), c='r')
plt.plot(r, np.percentile(der_onesigma, 84.13, axis=0), c='r')
         
plt.gca().set_xscale('log')
plt.gca().set_ylim([-10., 0])
plt.gca().set_xlim([.3, 10.])
plt.gca().set_xlabel('Comoving radius $r$ (Mpc)')
plt.gca().set_ylabel(r'$d\log \rho / d\log r$')
plt.show()



for i in xrange(n_samples):
    rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params[i, :]
    
    # Plot fraction of collapsed material 
    y_c = rho_Ein(r, [rho_s, r_s, logalpha])*f_trans(r, [r_t, logbeta, loggamma])
    logy_c = np.log(y_c)
    derivative_collapsed = np.gradient(logy_c, logr)
    
    plt.plot(r, derivative_collapsed, c='k', alpha=0.01)
    der_c_onesigma[i] = derivative_collapsed
    

y_plotn2 = -1.*(1.+3.*r/0.5)/(1.+r/0.5)
plt.plot(r, y_plotn2, '--b')
plt.axhline(-2.5, c='r', alpha=0.5)
plt.plot(r, np.percentile(der_c_onesigma, 15.86, axis=0), c='r')
plt.plot(r, np.percentile(der_c_onesigma, 84.13, axis=0), c='r')


plt.gca().set_xscale('log')
plt.gca().set_ylim([-10., 0])
plt.gca().set_xlim([.2, 10.])
plt.title('Collapsed derivative')
plt.show()





#Derivative of all material at splashback
plt.figure()
plt.hist(der[np.isfinite(der_c)], bins='auto', normed=True)
plt.gca().set_xlabel('derivative of all material at splashback')
plt.axvline(-2.5, c='r', ls='--')
#plt.gca().text()
plt.gca().set_xlim([-6,-2])
plt.show()
print "1 sigma limits:", np.percentile(der[np.isfinite(der_c)], 16), np.percentile(der[np.isfinite(der_c)], 50), np.percentile(der[np.isfinite(der_c)], 100-16)

print "3 sigma limits:", np.percentile(der[np.isfinite(der_c)], 0.135), np.percentile(der[np.isfinite(der_c)], 50), np.percentile(der[np.isfinite(der_c)], 100-0.135)



#Derivative of collapsed material at splashback
plt.figure()
plt.hist(der_c[np.isfinite(der_c)], bins='auto', normed=True)
plt.gca().set_xlabel('derivative of collapsed material at splashback')
plt.axvline(-2.5, c='r', ls='--')
#plt.gca().text()
plt.gca().set_xlim([-10,-2])
plt.show()
print "3 sigma limits:", np.percentile(der_c, 0.135), np.percentile(der_c, 50), np.percentile(der_c, 100-0.135)

#Splashback position
plt.figure()
plt.hist(r_sp[np.isfinite(der_c)], bins='auto', normed=True)
plt.gca().set_xlabel('splashback position')
plt.gca().set_xlim([0, 7])
plt.show()
print "one sigma limits:", np.percentile(r_sp, 15.85), np.percentile(r_sp, 50), np.percentile(r_sp, 100-15.85)


#Collapsed material at lower radius
plt.figure()
plt.hist(fraction_at_0[np.isfinite(der)], bins='auto', normed=True)
plt.gca().set_xlabel('fraction of collapsed material at lowest radius')
plt.show()